In [ ]:
import math

import numpy as np
import torch
import torch.nn as nn
import torchvision

from lightly.data import LightlyDataset
from lightly.loss import NegativeCosineSimilarity
from lightly.models.modules.heads import SimSiamPredictionHead, SimSiamProjectionHead
from lightly.transforms import SimCLRTransform, utils

In [ ]:
num_workers = 8
batch_size = 128
seed = 1
epochs = 50
input_size = 256

# dimension of the embeddings
num_ftrs = 512
# dimension of the output of the prediction and projection heads
out_dim = proj_hidden_dim = 512
# the prediction head uses a bottleneck architecture
pred_hidden_dim = 2

In [ ]:
# seed torch and numpy
torch.manual_seed(0)
np.random.seed(0)

# set the path to the dataset
path_to_data = '/home/plaiground/Documents/dataset/bodimgsplit/training/'
#path_to_data = "/home/plaiground/Documents/dataset/debinolabel/"

In [ ]:
# define the augmentations for self-supervised learning
transform = SimCLRTransform(
    input_size=input_size,
    # require invariance to flips and rotations
    hf_prob=0.5,
    vf_prob=0.5,
    rr_prob=0.5,
    # satellite images are all taken from the same height
    # so we use only slight random cropping
    min_scale=0.5,
    # use a weak color jitter for invariance w.r.t small color changes
    cj_prob=0.2,
    cj_bright=0.1,
    cj_contrast=0.1,
    cj_hue=0.1,
    cj_sat=0.1,
)

# create a lightly dataset for training with augmentations
dataset_train_simsiam = LightlyDataset(input_dir=path_to_data, transform=transform)

# create a dataloader for training
dataloader_train_simsiam = torch.utils.data.DataLoader(
    dataset_train_simsiam,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    num_workers=num_workers,
)

# create a torchvision transformation for embedding the dataset after training
# here, we resize the images to match the input size during training and apply
# a normalization of the color channel based on statistics from imagenet
test_transforms = torchvision.transforms.Compose(
    [
        torchvision.transforms.Resize((input_size, input_size)),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(
            mean=utils.IMAGENET_NORMALIZE["mean"],
            std=utils.IMAGENET_NORMALIZE["std"],
        ),
    ]
)

# create a lightly dataset for embedding
dataset_test = LightlyDataset(input_dir=path_to_data, transform=test_transforms)

# create a dataloader for embedding
dataloader_test = torch.utils.data.DataLoader(
    dataset_test,
    batch_size=batch_size,
    shuffle=False,
    drop_last=False,
    num_workers=num_workers,
)

In [ ]:
class SimSiam(nn.Module):
    def __init__(self, backbone, num_ftrs, proj_hidden_dim, pred_hidden_dim, out_dim):
        super().__init__()
        self.backbone = backbone
        self.projection_head = SimSiamProjectionHead(num_ftrs, proj_hidden_dim, out_dim)
        self.prediction_head = SimSiamPredictionHead(out_dim, pred_hidden_dim, out_dim)

    def forward(self, x):
        # get representations
        f = self.backbone(x).flatten(start_dim=1)
        # get projections
        z = self.projection_head(f)
        # get predictions
        p = self.prediction_head(z)
        # stop gradient
        z = z.detach()
        return z, p


# we use a pretrained resnet for this tutorial to speed
# up training time but you can also train one from scratch
resnet = torchvision.models.resnet18()
backbone = nn.Sequential(*list(resnet.children())[:-1])
model = SimSiam(backbone, num_ftrs, proj_hidden_dim, pred_hidden_dim, out_dim)

In [ ]:
# SimSiam uses a symmetric negative cosine similarity loss
criterion = NegativeCosineSimilarity()

# scale the learning rate
lr = 0.05 * batch_size / 256
# use SGD with momentum and weight decay
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

avg_loss = 0.0
avg_output_std = 0.0
for e in range(epochs):
    for (x0, x1), _, _ in dataloader_train_simsiam:
        # move images to the gpu
        x0 = x0.to(device)
        x1 = x1.to(device)

        # run the model on both transforms of the images
        # we get projections (z0 and z1) and
        # predictions (p0 and p1) as output
        z0, p0 = model(x0)
        z1, p1 = model(x1)

        # apply the symmetric negative cosine similarity
        # and run backpropagation
        loss = 0.5 * (criterion(z0, p1) + criterion(z1, p0))
        loss.backward()

        optimizer.step()
        optimizer.zero_grad()

        # calculate the per-dimension standard deviation of the outputs
        # we can use this later to check whether the embeddings are collapsing
        output = p0.detach()
        output = torch.nn.functional.normalize(output, dim=1)

        output_std = torch.std(output, 0)
        output_std = output_std.mean()

        # use moving averages to track the loss and standard deviation
        w = 0.9
        avg_loss = w * avg_loss + (1 - w) * loss.item()
        avg_output_std = w * avg_output_std + (1 - w) * output_std.item()

    # the level of collapse is large if the standard deviation of the l2
    # normalized output is much smaller than 1 / sqrt(dim)
    collapse_level = max(0.0, 1 - math.sqrt(out_dim) * avg_output_std)
    # print intermediate results
    print(
        f"[Epoch {e:3d}] "
        f"Loss = {avg_loss:.2f} | "
        f"Collapse Level: {collapse_level:.2f} / 1.00"
    )

OSError: Caught OSError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/plaiground/anaconda3/envs/ssl/lib/python3.11/site-packages/PIL/ImageFile.py", line 249, in load
    s = read(self.decodermaxblock)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/plaiground/anaconda3/envs/ssl/lib/python3.11/site-packages/PIL/PngImagePlugin.py", line 957, in load_read
    cid, pos, length = self.png.read()
                       ^^^^^^^^^^^^^^^
  File "/home/plaiground/anaconda3/envs/ssl/lib/python3.11/site-packages/PIL/PngImagePlugin.py", line 179, in read
    length = i32(s)
             ^^^^^^
  File "/home/plaiground/anaconda3/envs/ssl/lib/python3.11/site-packages/PIL/_binary.py", line 85, in i32be
    return unpack_from(">I", c, o)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^
struct.error: unpack_from requires a buffer of at least 4 bytes for unpacking 4 bytes at offset 0 (actual buffer size is 0)

The above exception was the direct cause of the following exc

In [ ]:
#skip
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import numpy as np
import pytorch_lightning as pl
import lightly
from pytorch_lightning.callbacks import TQDMProgressBar



num_workers = 8
max_epochs = 800
knn_k = 200
knn_t = 0.1
classes = 25
batch_size = 512
seed=1

pl.seed_everything(seed)

# use a GPU if available
gpus = 1 if torch.cuda.is_available() else 0

# Use SimCLR augmentations, additionally, disable blur
collate_fn = lightly.data.SimCLRCollateFunction(
    input_size=32,
    gaussian_blur=0.,
)

# No additional augmentations for the test set
test_transforms = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(
        mean=lightly.data.collate.imagenet_normalize['mean'],
        std=lightly.data.collate.imagenet_normalize['std'],
    )
])
#dataset_train_ssl = dataset_train_simsiam
dataset_train_ssl = dataset_train_simsiam
#    torchvision.datasets.CIFAR10(
#       root='data',
#      train=True))
#        download=True))
dataset_train_kNN = LightlyDataset(input_dir=path_to_data, 
#    root='data',
#     train=True,
     transform=test_transforms)
#    download=True))

dataset_test = dataset_test
#    root='data',
#   train=False,
#    transform=test_transforms))
    #download=True))

dataloader_train_ssl = torch.utils.data.DataLoader(
    dataset_train_ssl,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn,
    drop_last=True,
    num_workers=num_workers
)
dataloader_train_kNN = torch.utils.data.DataLoader(
    dataset_train_kNN,
    batch_size=batch_size,
    shuffle=False,
    drop_last=False,
    num_workers=num_workers
)
dataloader_test = torch.utils.data.DataLoader(
    dataset_test,
    batch_size=batch_size,
    shuffle=False,
    drop_last=False,
    num_workers=num_workers
)

# code for kNN prediction from here:
# https://colab.research.google.com/github/facebookresearch/moco/blob/colab-notebook/colab/moco_cifar10_demo.ipynb
def knn_predict(feature, feature_bank, feature_labels, classes: int, knn_k: int, knn_t: float):
    """Helper method to run kNN predictions on features based on a feature bank

    Args:
        feature: Tensor of shape [N, D] consisting of N D-dimensional features
        feature_bank: Tensor of a database of features used for kNN
        feature_labels: Labels for the features in our feature_bank
        classes: Number of classes (e.g. 10 for CIFAR-10)
        knn_k: Number of k neighbors used for kNN
        knn_t: 

    """
    # compute cos similarity between each feature vector and feature bank ---> [B, N]
    sim_matrix = torch.mm(feature, feature_bank)
    # [B, K]
    sim_weight, sim_indices = sim_matrix.topk(k=knn_k, dim=-1)
    # [B, K]
    sim_labels = torch.gather(feature_labels.expand(feature.size(0), -1), dim=-1, index=sim_indices)
    # we do a reweighting of the similarities 
    sim_weight = (sim_weight / knn_t).exp()
    # counts for each class
    one_hot_label = torch.zeros(feature.size(0) * knn_k, classes, device=sim_labels.device)
    # [B*K, C]
    one_hot_label = one_hot_label.scatter(dim=-1, index=sim_labels.view(-1, 1), value=1.0)
    # weighted score ---> [B, C]
    pred_scores = torch.sum(one_hot_label.view(feature.size(0), -1, classes) * sim_weight.unsqueeze(dim=-1), dim=1)
    pred_labels = pred_scores.argsort(dim=-1, descending=True)
    return pred_labels


class BenchmarkModule(pl.LightningModule):
    """A PyTorch Lightning Module for automated kNN callback
    
    At the end of every training epoch we create a feature bank by inferencing
    the backbone on the dataloader passed to the module. 
    At every validation step we predict features on the validation data.
    After all predictions on validation data (validation_epoch_end) we evaluate
    the predictions on a kNN classifier on the validation data using the 
    feature_bank features from the train data.
    We can access the highest accuracy during a kNN prediction using the 
    max_accuracy attribute.
    """
    def __init__(self, dataloader_kNN):
        super().__init__()
        self.backbone = nn.Module()
        self.max_accuracy = 0.0
        self.dataloader_kNN = dataloader_kNN

    def training_epoch_end(self, outputs):
        # update feature bank at the end of each training epoch
        self.backbone.eval()
        self.feature_bank = []
        self.targets_bank = []
        with torch.no_grad():
            for data in self.dataloader_kNN:
                img, target, _ = data
                if gpus > 0:
                    img = img.cuda()
                    target = target.cuda()
                feature = self.backbone(img).squeeze()
                feature = F.normalize(feature, dim=1)
                self.feature_bank.append(feature)
                self.targets_bank.append(target)
        self.feature_bank = torch.cat(self.feature_bank, dim=0).t().contiguous()
        self.targets_bank = torch.cat(self.targets_bank, dim=0).t().contiguous()
        self.backbone.train()

    def validation_step(self, batch, batch_idx):
        # we can only do kNN predictions once we have a feature bank
        if hasattr(self, 'feature_bank') and hasattr(self, 'targets_bank'):
            images, targets, _ = batch
            feature = self.backbone(images).squeeze()
            feature = F.normalize(feature, dim=1)
            pred_labels = knn_predict(feature, self.feature_bank, self.targets_bank, classes, knn_k, knn_t)
            num = images.size(0)
            top1 = (pred_labels[:, 0] == targets).float().sum().item()
            return (num, top1)
    
    def validation_epoch_end(self, outputs):
        if outputs:
            total_num = 0
            total_top1 = 0.
            for (num, top1) in outputs:
                total_num += num
                total_top1 += top1
            acc = float(total_top1 / total_num)
            if acc > self.max_accuracy:
                self.max_accuracy = acc
            self.log('kNN_accuracy', acc * 100.0, prog_bar=True)


class SimSiamModel(BenchmarkModule):
    def __init__(self, dataloader_kNN):
        super().__init__(dataloader_kNN)
        # create a ResNet backbone and remove the classification head
        resnet = lightly.models.ResNetGenerator('resnet-18')
        self.backbone = nn.Sequential(
            *list(resnet.children())[:-1],
            nn.AdaptiveAvgPool2d(1),
        )
        # create a simsiam model based on ResNet
        self.resnet_simsiam = \
            lightly.models.SimSiam(self.backbone, num_ftrs=512, out_dim=2)
        self.criterion = lightly.loss.SymNegCosineSimilarityLoss()
            
    def forward(self, x):
        self.resnet_simsiam(x)

    def training_step(self, batch, batch_idx):
        (x0, x1), _, _ = batch
        x0, x1 = self.resnet_simsiam(x0, x1)
        loss = self.criterion(x0, x1)
        self.log('train_loss_ssl', loss)
        return loss

    def configure_optimizers(self):
        optim = torch.optim.SGD(self.resnet_simsiam.parameters(), lr=6e-2,
                                momentum=0.9, weight_decay=5e-4)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optim, max_epochs)
        return [optim], [scheduler]


model = SimSiamModel(dataloader_train_kNN)
trainer = pl.Trainer(max_epochs=max_epochs, accelerator="gpu",
                    callbacks=[TQDMProgressBar(refresh_rate=100)])
trainer.fit(
    model,
    train_dataloader=dataloader_train_ssl,
    val_dataloaders=dataloader_test
)

print(f'Highest test accuracy: {model.max_accuracy:.4f}')

Global seed set to 1
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


TypeError: Trainer.fit() got an unexpected keyword argument 'train_dataloader'

In [ ]:
#testing unit color from moco classifier: ini datasetnya saya split dulu 80,20 (training test)
import copy
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torchvision
from lightly.data import LightlyDataset
from lightly.loss import NTXentLoss
from lightly.models.utils import deactivate_requires_grad                        
from lightly.models.utils import update_momentum
from lightly.models.utils import batch_shuffle
from lightly.models.utils import batch_unshuffle
from lightly.models import ResNetGenerator
from lightly.models.modules.heads import MoCoProjectionHead
from lightly.transforms import MoCoV2Transform, utils
from pytorch_lightning.callbacks import TQDMProgressBar
#path_to_train = '/home/ismail/datasets/debisplit/training'
#path_to_test = '/home/ismail/datasets/debisplit/test'
num_workers = 8
batch_size=512
memory_bank_size=4096
seed = 1
max_epochs=300
path_to_train = '/home/plaiground/Documents/dataset/debi872split/training/'
path_to_test = '/home/plaiground/Documents/dataset/debi872split/test/'


In [ ]:
#cnnimage
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models, transforms, datasets
from PIL import Image
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
#use data augmentation
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
])


In [ ]:
# Define a simple CNN model
class SpectrogramClassifier(nn.Module):
    def __init__(self, num_classes):
        super(SpectrogramClassifier, self).__init__()
        #self.model = models.resnet18(pretrained=True)
        #self.model = models.resnet18(pretrained=False)
        self.model = models.resnet18(weights=None)
        self.model.load_state_dict(torch.load('/home/plaiground/Documents/resnet-18/resnet18-f37072fd.pth'))
        in_features = self.model.fc.in_features
        self.model.fc = nn.Linear(in_features, num_classes)

    def forward(self, x):
        return self.model(x)
        #x = self.model(x)  # Pass through pre-trained model (ResNet-18)
        # Print feature shape before final layer
        #print(f"Spectrogram feature shape before final layer: {x.shape}")
        #x = self.fc(x)  # Final classification layer
        #return x

# Custom dataset class
class SpectrogramDataset(torch.utils.data.Dataset):
    def __init__(self, root, transform=None):
        self.root = root
        self.transform = transform
        self.classes = sorted(os.listdir(root))
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}
        self.img_list = self.generate_img_list()

    def generate_img_list(self):
        img_list = []
        for cls in self.classes:
            cls_folder = os.path.join(self.root, cls)
            for img_name in os.listdir(cls_folder):
                img_list.append((os.path.join(cls_folder, img_name), cls))
        return img_list

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, index):
        img_path, cls = self.img_list[index]
        img = Image.open(img_path).convert('RGB')

        if self.transform is not None:
            img = self.transform(img)

        return img, self.class_to_idx[cls]

In [ ]:
# Define transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])
root_folder = '/home/plaiground/Documents/dataset/maldeb/'
dataset = SpectrogramDataset(root_folder, transform=transform)
train_dataset=SpectrogramDataset(root_folder, transform=train_transform)
train_loader=DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
# Set up datasets and dataloaders
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Initialize the model, loss function, and optimizer
num_classes = len(dataset.classes)
model = SpectrogramClassifier(num_classes=2)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Train the model
num_epochs = 100
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Evaluate the model on the test set
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    print(f'Epoch [{epoch + 1}/{num_epochs}], Accuracy: {accuracy:.2f}')

Epoch [1/100], Accuracy: 0.95
Epoch [2/100], Accuracy: 0.96
Epoch [3/100], Accuracy: 0.97
Epoch [4/100], Accuracy: 0.97
Epoch [5/100], Accuracy: 0.97
Epoch [6/100], Accuracy: 0.97
Epoch [7/100], Accuracy: 0.97
Epoch [8/100], Accuracy: 0.97
Epoch [9/100], Accuracy: 0.97
Epoch [10/100], Accuracy: 0.96
Epoch [11/100], Accuracy: 0.98
Epoch [12/100], Accuracy: 0.98
Epoch [13/100], Accuracy: 0.98
Epoch [14/100], Accuracy: 0.98
Epoch [15/100], Accuracy: 0.98
Epoch [16/100], Accuracy: 0.98
Epoch [17/100], Accuracy: 0.98
Epoch [18/100], Accuracy: 0.97
Epoch [19/100], Accuracy: 0.97
Epoch [20/100], Accuracy: 0.98
Epoch [21/100], Accuracy: 0.98
Epoch [22/100], Accuracy: 0.98
Epoch [23/100], Accuracy: 0.98
Epoch [24/100], Accuracy: 0.98
Epoch [25/100], Accuracy: 0.97
Epoch [26/100], Accuracy: 0.98
Epoch [27/100], Accuracy: 0.98
Epoch [28/100], Accuracy: 0.98
Epoch [29/100], Accuracy: 0.98
Epoch [30/100], Accuracy: 0.98
Epoch [31/100], Accuracy: 0.98
Epoch [32/100], Accuracy: 0.98
Epoch [33/100], A

In [ ]:
# Save the trained model
torch.save(model.state_dict(), 'imgcnnmaldeb.pth')

In [ ]:
#spectro
# Define transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])
root_folder = '/home/plaiground/Documents/dataset/maldebspectro/'
dataset = SpectrogramDataset(root_folder, transform=transform)
train_dataset=SpectrogramDataset(root_folder, transform=train_transform)
train_loader=DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
# Set up datasets and dataloaders
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Initialize the model, loss function, and optimizer
num_classes = len(dataset.classes)
model = SpectrogramClassifier(num_classes=2)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Train the model
num_epochs = 100
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Evaluate the model on the test set
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    print(f'Epoch [{epoch + 1}/{num_epochs}], Accuracy: {accuracy:.2f}')

/home/plaiground/anaconda3/envs/ssl/lib/python3.11/site-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (92618880 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Epoch [1/100], Accuracy: 0.82


/home/plaiground/anaconda3/envs/ssl/lib/python3.11/site-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (92618880 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Epoch [2/100], Accuracy: 0.87


/home/plaiground/anaconda3/envs/ssl/lib/python3.11/site-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (92618880 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


...

Epoch [98/100], Accuracy: 0.96


/home/plaiground/anaconda3/envs/ssl/lib/python3.11/site-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (92618880 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Epoch [99/100], Accuracy: 0.96


/home/plaiground/anaconda3/envs/ssl/lib/python3.11/site-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (92618880 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Epoch [100/100], Accuracy: 0.96


In [ ]:
# Save the trained model
torch.save(model.state_dict(), 'spectroCNNmaldeb.pth')

In [ ]:
#gemini late-fusion-average
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import models, transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Dataset
import pytorch_lightning as pl
from sklearn.metrics import accuracy_score
import numpy as np


class ImageClassifier(pl.LightningModule):
    def __init__(self,num_classes=2):
        super(ImageClassifier, self).__init__()
        # Assuming your SimCLR model has a final projection head
        self.model = models.resnet18(weights=None)
        #torch.load(pretrained_model_path)
        #self.classifier = nn.Linear(in_features=(1, 224, 224), out_features=2)  # Replace ... with actual input size
        #resnet = torch.hub.load('pytorch/vision:v0.13.1', 'resnet18', pretrained=True)  # Load pre-trained ResNet
        #resnet = torchvision.models.resnet18(pretrained=False)
        #resnet = models.resnet18(pretrained=False)
        #resnet = models.resnet18(weights=None)
        #self.backbone = nn.Sequential(*list(resnet.children())[:-1])  # Extract backbone
        #hidden_dim = resnet.fc.in_features
        #self.classification_head = nn.Linear(hidden_dim, 2)
        self.model.load_state_dict(torch.load('/home/plaiground/Documents/resnet-18/resnet18-f37072fd.pth'))
        in_features = self.model.fc.in_features
        self.model.fc = nn.Linear(in_features, num_classes)
    
    def forward(self, x):
        #x = self.model(x)  # Use the entire loaded model for feature extraction
        #x = self.classifier(x)
        #return x
        #h = self.backbone(x).flatten(start_dim=1)
        #eturn self.classification_head(h)
        return self.model(x)

#class SpectrogramCNN(nn.Module):
class SpectrogramClassifier(nn.Module):
    #def __init__(self, model_path):
    def __init__(self, num_classes=2):
        super(SpectrogramClassifier, self).__init__()
        #self.model = models.resnet18(pretrained=False)
        self.model = models.resnet18(weights=None)
        self.model.load_state_dict(torch.load('/home/plaiground/Documents/resnet-18/resnet18-f37072fd.pth'))
        in_features = self.model.fc.in_features
        self.model.fc = nn.Linear(in_features, num_classes)
        #self.model = torch.load(model_path)  # Load the entire pre-trained spectrogram CNN

    def forward(self, x):
        return self.model(x)

# Define ImageClassifier wrapper
class ImageClassifierWrapper(nn.Module):
    def __init__(self, pretrained_model_path, num_classes=2):
        super(ImageClassifierWrapper, self).__init__()
        self.model = ImageClassifier(num_classes)
        self.load_pretrained_weights(pretrained_model_path)

    def load_pretrained_weights(self, pretrained_model_path):
        state_dict = torch.load(pretrained_model_path)
        self.model.load_state_dict(state_dict, strict=False)

    def forward(self, x):
        return self.model(x)

# Define SpectrogramClassifierWrapper
class SpectrogramClassifierWrapper(nn.Module):
    def __init__(self, pretrained_model_path, num_classes=2):
        super(SpectrogramClassifierWrapper, self).__init__()
        self.model = SpectrogramClassifier(num_classes)
        self.load_pretrained_weights(pretrained_model_path)

    def load_pretrained_weights(self, pretrained_model_path):
        state_dict = torch.load(pretrained_model_path)
        self.model.load_state_dict(state_dict, strict=False)

    def forward(self, x):
        return self.model(x)

class MultimodalLateFusion(nn.Module):
    def __init__(self, image_model_path, spectrogram_model_path):
        super(MultimodalLateFusion, self).__init__()
        self.image_model = ImageModel(image_model_path)
        self.spectrogram_model = SpectrogramClassifier(spectrogram_model_path)
        #image_features = self.image_model(image_sample)
        #spectrogram_features = self.spectrogram_model(spectrogram_sample)
        #print(f"Image feature shape: {image_features.shape}")
        #print(f"Spectrogram feature shape: {spectrogram_features.shape}")
        model = SpectrogramClassifier(num_classes=2)  # Replace 2 with your actual number of classes
        spectrogram_dim = model.model.fc.in_features  # Assuming fc is the final layer
        combined_feature_size = 128 + spectrogram_dim
        self.fusion_layer = nn.Linear(combined_feature_size, 2)  # Replace ... with combined feature size

    def forward(self, image, spectrogram):
        image_features = self.image_model(image)
        spectrogram_features = self.spectrogram_model(spectrogram)
        # Concatenate features along channel dimension (dim=1)
        combined_features = torch.cat((image_features, spectrogram_features), dim=1)
        output = self.fusion_layer(combined_features)
        return output

# Define transforms for the datasets
data_transforms = {
    'images': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ]),
    'spectrograms': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])
}

# Load the datasets
image_dataset = ImageFolder(root='/home/plaiground/Documents/dataset/maldeb/', transform=data_transforms['images'])
spectrogram_dataset = ImageFolder(root='/home/plaiground/Documents/dataset/maldebspectro/', transform=data_transforms['spectrograms'])

# Create a custom dataset to pair images and spectrograms
class PairedDataset(Dataset):
    def __init__(self, image_dataset, spectrogram_dataset):
        self.image_dataset = image_dataset
        self.spectrogram_dataset = spectrogram_dataset

    def __len__(self):
        return len(self.image_dataset)

    def __getitem__(self, idx):
        image, label = self.image_dataset[idx]
        spectrogram, _ = self.spectrogram_dataset[idx]
        return image, spectrogram, label

# Create the paired dataset and dataloader
paired_dataset = PairedDataset(image_dataset, spectrogram_dataset)
test_loader = DataLoader(paired_dataset, batch_size=32, shuffle=False)

# Example usage
image_model_path = "/home/plaiground/Documents/multimodal/imgcnnmaldeb.pth"
spectrogram_model_path = "/home/plaiground/Documents/multimodal/spectroCNNmaldeb.pth"

#image_model = SimCLRModel(image_model_path)
#spectrogram_model = SpectrogramCNN(spectrogram_model_path)

image_model = ImageClassifierWrapper(image_model_path)
spectrogram_model = SpectrogramClassifierWrapper(spectrogram_model_path)

# Set models to evaluation mode
image_model.eval()
spectrogram_model.eval()
# Define your evaluation metric function (replace with your chosen metric)

def evaluate(model, data_loader):
    model.eval()
    total_correct = 0
    total_samples = 0
    with torch.no_grad():
        #for images, spectrograms, labels in data_loader:
            #images = images.to('cuda' if torch.cuda.is_available() else 'cpu')
            #spectrograms = spectrograms.to('cuda' if torch.cuda.is_available() else 'cpu')
        for data, labels in data_loader:
            data = data.to('cuda' if torch.cuda.is_available() else 'cpu')
            labels = labels.to('cuda' if torch.cuda.is_available() else 'cpu')

            # Pass data through the late fusion model
            #outputs = fusion_model(images, spectrograms)
            #outputs = fusion_model(data)  # Assuming both data sources are combined within the model
            outputs = model(data)
            _, predictions = torch.max(outputs, 1)
            total_correct += (predictions == labels).sum().item()
            total_samples += labels.size(0)
    accuracy = total_correct / total_samples
    return accuracy

# Function to get predictions from both models
def get_predictions(image_tensor, spectrogram_tensor):
    with torch.no_grad():
        image_output = image_model(image_tensor)
        spectrogram_output = spectrogram_model(spectrogram_tensor)
    
    # Apply softmax to get probabilities
    image_probs = F.softmax(image_output, dim=1)
    spectrogram_probs = F.softmax(spectrogram_output, dim=1)
    
    return image_probs, spectrogram_probs

# Function to perform late fusion and make final decision
def late_fusion(image_tensor, spectrogram_tensor):
    image_probs, spectrogram_probs = get_predictions(image_tensor, spectrogram_tensor)
    
    # Combine the probabilities (simple average here, can use other methods)
    combined_probs = (image_probs + spectrogram_probs) / 2
    
    # Final prediction
    _, final_pred = torch.max(combined_probs, 1)
    
    return final_pred
# Evaluate the model on the test set
#accuracy = evaluate(fusion_model, test_loader)
#print(f"Test Accuracy: {accuracy:.4f}")

# Evaluate the model on the test sets
#spectrogram_accuracy = evaluate(spectrogram_model, spectrogram_test_loader)
#simclr_accuracy = evaluate(simclr_model, simclr_test_loader)
# Function to evaluate accuracy on the test dataset
def evaluate_model(image_model, spectrogram_model, test_loader):
    image_model.eval()
    spectrogram_model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, spectrograms, labels in test_loader:
            images = images.to(device)
            spectrograms = spectrograms.to(device)
            labels = labels.to(device)

            preds = late_fusion(images, spectrograms)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    return accuracy


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
image_model.to(device)
spectrogram_model.to(device)

# Evaluate the model
accuracy = evaluate_model(image_model, spectrogram_model, test_loader)
print(f'Test Accuracy: {accuracy:.4f}')


/home/plaiground/anaconda3/envs/ssl/lib/python3.11/site-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (92618880 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Test Accuracy: 0.9971


In [19]:
#weighted average
image_model.eval()
spectrogram_model.eval()
# Function to get predictions from both models
def get_predictions(image_tensor, spectrogram_tensor):
    with torch.no_grad():
        image_output = image_model(image_tensor)
        spectrogram_output = spectrogram_model(spectrogram_tensor)
    
    # Apply softmax to get probabilities
    image_probs = F.softmax(image_output, dim=1)
    spectrogram_probs = F.softmax(spectrogram_output, dim=1)
    
    return image_probs, spectrogram_probs

def late_fusion_weighted(image_tensor, spectrogram_tensor, image_weight=0.5, spectrogram_weight=0.5):
    image_probs, spectrogram_probs = get_predictions(image_tensor, spectrogram_tensor)
    
    # Ensure weights sum to 1
    total_weight = image_weight + spectrogram_weight
    image_weight /= total_weight
    spectrogram_weight /= total_weight
    
    # Weighted average of the probabilities
    combined_probs = image_weight * image_probs + spectrogram_weight * spectrogram_probs
    
    # Final prediction
    _, final_pred = torch.max(combined_probs, 1)
    
    return final_pred

# Function to evaluate accuracy on the test dataset with weighted fusion
def evaluate_model_weighted(image_model, spectrogram_model, test_loader, image_weight=0.5, spectrogram_weight=0.5):
    image_model.eval()
    spectrogram_model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, spectrograms, labels in test_loader:
            images = images.to(device)
            spectrograms = spectrograms.to(device)
            labels = labels.to(device)

            preds = late_fusion_weighted(images, spectrograms, image_weight, spectrogram_weight)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    return accuracy
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
image_model.to(device)
spectrogram_model.to(device)

# Evaluate the model with weighted average fusion
accuracy_weighted = evaluate_model_weighted(image_model, spectrogram_model, test_loader, image_weight=0.6, spectrogram_weight=0.4)
print(f'Test Accuracy (Weighted Fusion): {accuracy_weighted:.4f}')

/home/plaiground/anaconda3/envs/ssl/lib/python3.11/site-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (92618880 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Test Accuracy (Weighted Fusion): 0.9971


In [20]:
#logreg fusion

from sklearn.linear_model import LogisticRegression
image_model.eval()
spectrogram_model.eval()
# Function to get predictions from both models
def get_predictions(image_tensor, spectrogram_tensor):
    with torch.no_grad():
        image_output = image_model(image_tensor)
        spectrogram_output = spectrogram_model(spectrogram_tensor)
    
    # Apply softmax to get probabilities
    image_probs = F.softmax(image_output, dim=1)
    spectrogram_probs = F.softmax(spectrogram_output, dim=1)
    
    return image_probs, spectrogram_probs

def extract_features(image_model, spectrogram_model, dataloader):
    image_model.eval()
    spectrogram_model.eval()
    features = []
    labels = []

    with torch.no_grad():
        for images, spectrograms, label in dataloader:
            images = images.to(device)
            spectrograms = spectrograms.to(device)
            label = label.to(device)

            image_output = image_model(images)
            spectrogram_output = spectrogram_model(spectrograms)

            image_probs = F.softmax(image_output, dim=1)
            spectrogram_probs = F.softmax(spectrogram_output, dim=1)

            combined_features = torch.cat((image_probs, spectrogram_probs), dim=1)
            features.append(combined_features.cpu().numpy())
            labels.append(label.cpu().numpy())

    return np.vstack(features), np.hstack(labels)

# Extract features from the training set
train_features, train_labels = extract_features(image_model, spectrogram_model, test_loader)

# Train a logistic regression model on the extracted features
log_reg = LogisticRegression(max_iter=100000)
log_reg.fit(train_features, train_labels)

# Function to evaluate the logistic regression model
def evaluate_log_reg(log_reg, image_model, spectrogram_model, test_loader):
    features, labels = extract_features(image_model, spectrogram_model, test_loader)
    preds = log_reg.predict(features)
    accuracy = accuracy_score(labels, preds)
    return accuracy

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
image_model.to(device)
spectrogram_model.to(device)


# Evaluate the logistic regression model
accuracy_log_reg = evaluate_log_reg(log_reg, image_model, spectrogram_model, test_loader)
print(f'Test Accuracy (Logistic Regression Fusion): {accuracy_log_reg:.4f}')

/home/plaiground/anaconda3/envs/ssl/lib/python3.11/site-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (92618880 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/home/plaiground/anaconda3/envs/ssl/lib/python3.11/site-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (92618880 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Test Accuracy (Logistic Regression Fusion): 0.9974


In [ ]:
# Export Image Model to ONNX untuk Desktop App
# Model: Image CNN (imgcnnmaldeb) - Single Model

import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models

class ImageClassifier(nn.Module):
    def __init__(self, num_classes=2):
        super(ImageClassifier, self).__init__()
        self.model = models.resnet18(weights=None)
        in_features = self.model.fc.in_features
        self.model.fc = nn.Linear(in_features, num_classes)
    
    def forward(self, x):
        return self.model(x)

# Load trained model
image_model_path = "C:\\Users\\saefu\\Documents\\Mango-app-almost\\frontend\\models\\imgcnnmaldeb.pth"

# Create model instance
model = ImageClassifier(num_classes=2)

# Load trained weights
model.load_state_dict(torch.load(image_model_path, map_location='cpu'), strict=False)
model.eval()

print(f"Model loaded: {image_model_path}")

# Create dummy input
dummy_input = torch.randn(1, 3, 224, 224)

# Path output
output_dir = "desktop_app/models"
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "Modelv3.onnx")

print(f"\n🔄 Exporting to ONNX...")

# Export to ONNX
torch.onnx.export(
    model,
    dummy_input,
    output_path,
    export_params=True,
    opset_version=18,
    do_constant_folding=True,
    input_names=['input'],
    output_names=['output'],
    dynamic_axes={
        'input': {0: 'batch_size'},
        'output': {0: 'batch_size'}
    },
    dynamo=False,
    verbose=False
)

print(f"\nModel berhasil diekspor!")
print(f"   Output: {output_path}")

# Get file size
file_size = os.path.getsize(output_path) / (1024 * 1024)
print(f"   Size: {file_size:.2f} MB")

🥭 MangoDefend - Export Image Model to ONNX

📦 Loading image model...
   ✅ Model loaded: C:\Users\saefu\Documents\Mango-app-almost\frontend\models\imgcnnmaldeb.pth

🔄 Exporting to ONNX...
   Input: [batch, 3, 224, 224]
   Output: [batch, 2] (class probabilities)


C:\Users\saefu\AppData\Local\Temp\ipykernel_39788\2572175975.py:56: DeprecationWarning: You are using the legacy TorchScript-based ONNX export. Starting in PyTorch 2.9, the new torch.export-based ONNX exporter will be the default. To switch now, set dynamo=True in torch.onnx.export. This new exporter supports features like exporting LLMs with DynamicCache. We encourage you to try it and share feedback to help improve the experience. Learn more about the new export logic: https://pytorch.org/docs/stable/onnx_dynamo.html. For exporting control flow: https://pytorch.org/tutorials/beginner/onnx/export_control_flow_model_to_onnx_tutorial.html.
  torch.onnx.export(



✅ Model berhasil diekspor!
   Output: desktop_app/models\Modelv3.onnx
   Size: 42.63 MB

🔍 Verifying ONNX model...
   ✅ Model ONNX valid!

📋 Model Information:
   Producer: pytorch
   IR Version: 8
   Opset: 18

✅ Export selesai! Model siap digunakan di desktop_app

📝 Cara menggunakan di desktop app:
   import onnxruntime as ort
   session = ort.InferenceSession('desktop_app/models/mango_image_model.onnx')
   outputs = session.run(None, {'input': image_array})
   probabilities = outputs[0]  # [benign_prob, malware_prob]


: 

In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from sklearn.metrics import accuracy_score

class DeepFusionModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(DeepFusionModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

#def get_label(path):
"""
#  Extracts label from the folder name of an image path.
#
#  Args:
#      path (str): Path to the image file.

#  Returns:
#      str: Label extracted from the folder name.
"""
  # Split the path based on the directory separator
#  folder_name = path.split("/")[-1]  # Assuming "/" as separator
#  return folder_name


# Initialize models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#image_model_path = '/path/to/image_model.pth'
#spectrogram_model_path = '/path/to/spectrogram_model.pth'
image_model_path = "/home/plaiground/Documents/multimodal/imgcnndebi872a.pth"
spectrogram_model_path = "/home/plaiground/Documents/multimodal/spectroCNNdebi872.pth"

image_model = ImageClassifierWrapper(image_model_path)
#image_model_state = torch.load(image_model_path)
image_model.to(device)

# Filter out unnecessary keys
filtered_image_model_state = {k: v for k, v in image_model_state.items() if k in image_model.state_dict()}
image_model.load_state_dict(filtered_image_model_state, strict=False)
image_model.to(device)

spectrogram_model = SpectrogramClassifierWrapper(spectrogram_model_path)
spectrogram_model.to(device)

# Fusion model
input_dim = 512 * 4 * 4 + 512 * 4 * 4
hidden_dim = 128
output_dim = 2

fusion_model = DeepFusionModel(input_dim, hidden_dim, output_dim)
fusion_model.to(device)

# Training function
def train_fusion_model(fusion_model, image_model, spectrogram_model, train_loader, num_epochs=10, learning_rate=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(fusion_model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        fusion_model.train()
        running_loss = 0.0

        for images, spectrograms in train_loader:
            images, spectrograms = images.to(device), spectrograms.to(device)

            # Extract features
            image_features = image_model.backbone(images)
            spectrogram_features = spectrogram_model.backbone(spectrograms)

            # Flatten features
            image_features = torch.flatten(image_features, start_dim=1)
            spectrogram_features = torch.flatten(spectrogram_features, start_dim=1)

            # Concatenate features
            combined_features = torch.cat((image_features, spectrogram_features), dim=1)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward + backward + optimize
            outputs = fusion_model(combined_features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # Print statistics
            running_loss += loss.item()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

# Train the fusion model
train_fusion_model(fusion_model, image_model, spectrogram_model, train_loader)

# Evaluation function
def evaluate_fusion_model(fusion_model, image_model, spectrogram_model, test_loader):
    fusion_model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, spectrograms, labels in test_loader:
            images, spectrograms, labels = images.to(device), spectrograms.to(device), labels.to(device)

            # Extract features
            image_features = image_model.backbone(images)
            spectrogram_features = spectrogram_model.backbone(spectrograms)

            # Flatten features
            image_features = torch.flatten(image_features, start_dim=1)
            spectrogram_features = torch.flatten(spectrogram_features, start_dim=1)

            # Concatenate features
            combined_features = torch.cat((image_features, spectrogram_features), dim=1)

            # Forward pass
            outputs = fusion_model(combined_features)
            _, preds = torch.max(outputs, 1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    return accuracy

# Evaluate the fusion model
accuracy_fusion = evaluate_fusion_model(fusion_model, image_model, spectrogram_model, test_loader)
print(f'Test Accuracy (Deep Fusion): {accuracy_fusion:.4f}')

AttributeError: 'ImageClassifierWrapper' object has no attribute 'backbone'